In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from torch.optim import Adam
import torch.nn.functional as F
import pandas as pd
import numpy as np
import os
import sys

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/673Project/Datasets/Symptom2Disease.csv'
df = pd.read_csv(file_path)
print(len(df))
#print(df.head())
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
#print(shuffled_df.head())

1200


In [ ]:
X = list(df['text'])
Y = df['label']
print(X)
#print(Y)
#print(set(Y))

['I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches.', 'My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation.', 'I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints.', 'There is a silver like dusting on my skin, especially on my lower back and scalp. This dusting is made up of small scales that flake off easily when I scratch them.', 'My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms.', 'The skin on my palms and soles is thickened and has deep cracks. These cracks are painful and bleed easily.', 'The skin around my mouth, nose, and eyes is red and inflamed. It is often itchy and uncomfortable. There is a noticeable inflammation in

In [ ]:
unique_labels = list(set(Y))

one_hot_encoded = np.zeros((len(Y), len(unique_labels)))
print(type(one_hot_encoded[0][0]))
for i, label in enumerate(Y):

    one_hot_encoded[i][unique_labels.index(label)] = 1


<class 'numpy.float64'>


In [ ]:
#create train and test set
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_encoded, test_size=0.2, random_state=42)


In [ ]:
# Load ClinicalBERT
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
class ClinicalDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Tokenize text
        inputs = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.float),
            #"label_index":torch.tensor(label, dtype=torch.long),
        }



# Create dataset and DataLoader
dataset = ClinicalDataset(X_train, y_train, tokenizer)
#print(dataset[0])
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

{'input_ids': tensor([  101,   178,   112,  1396,  1151,  2385,  1122,  8992,  3055,   117,
         1105,   178,  1138,   187, 10733,  1183, 14879,  1155,  1166,  1139,
         2241,   119,  1175,  1132,  1145,  2218,  4001,  1115,  1132,  9934,
         1107,  5922,  1190,  1103,  1832,  1104,  1139,  2241,   117,  1105,
          178,   112,  1396,  1400,  1199,  3016, 16401,  1116,   119,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [ ]:
#crete testing dataset
dataset_test = ClinicalDataset(X_test, y_test, tokenizer)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=True)

true_labels_test = np.argmax(y_test, axis=1)
print(true_labels_test)
actual_labels_text = [unique_labels[i] for i in true_labels_test]


[17 15  6 11 22  8 19 13 11 15 18  7  7 18 10 21 20  3 18 11  0  4  5 19
  6  1  5  9  0  9  0 15  5 16  0  9 11 22 19  5  2  6  8  0 13 19  0  7
  7 10 16  8  0 21  8 19 13 17 12  0 19 21 10 22 21 13 22 11 20 23 11  6
 17 22 22 11 21 22 14 15 17 10 18  6 20 12  2 21 12  4 20 23  1  0 12  5
 20 14  1  6  9  6 16 22 15 16  8 14 17 14  0 15 21 13  4 12 10 21 16 21
 20 14 12  2 11 17 19  6  5  4  1  8 19 10 12  4  4 23 23 14  2  4  0  4
  3 20 16  5  9 23  8 19 16 10 14 11 19  1  8 19  2 14 12  1  1 18 22 15
 18  7  3 13 19  1 21  9 21  8 22  3 16 18 10 12  1 13 23 18 11 20 18  5
  7 18 23 21 22 14 13  1 18 14  5 13 15 10  3 16 16  8 17  3 11 15 14  1
  4 10  5  9  8 13 12 13  1 20  3 22 10  4 19  6 12  4 13  0 15 20 17 15]


In [ ]:
print(dataloader_test)

In [ ]:
class ClinicalBERTClassifier(nn.Module):
    def __init__(self, clinical_bert, num_classes):
        super(ClinicalBERTClassifier, self).__init__()
        self.bert = clinical_bert
        self.dropout = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        # Pass inputs through ClinicalBERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # Extract the [CLS] token representation
        cls_output = outputs.last_hidden_state[:, 0, :]
        # Apply dropout and classification layer
        logits = self.classifier(self.dropout(cls_output))
        logits = F.softmax(logits, dim=1)
        #return self.bert.config.hidden_size
        #logits = self.classifier(cls_output)
        return logits



In [ ]:
num_classes = len(unique_labels)
num_epochs = 13;
batch_size = 16
learning_rate = 2e-5
model_classifier = ClinicalBERTClassifier(model, num_classes)
optimizer = Adam(model_classifier.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(num_epochs):
    model_classifier.train()
    total_loss = 0
    for batch in dataloader:

        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"]
        # Forward pass
        optimizer.zero_grad()
        logits = model_classifier(input_ids, attention_mask)
        loss = criterion(logits, labels)
        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        dbg = 1
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")


Epoch 1/13, Loss: 3.1646
Epoch 2/13, Loss: 3.0278
Epoch 3/13, Loss: 2.8204
Epoch 4/13, Loss: 2.6592
Epoch 5/13, Loss: 2.5240
Epoch 6/13, Loss: 2.4239
Epoch 7/13, Loss: 2.3695
Epoch 8/13, Loss: 2.3260
Epoch 9/13, Loss: 2.3050
Epoch 10/13, Loss: 2.2805
Epoch 11/13, Loss: 2.2664
Epoch 12/13, Loss: 2.2597
Epoch 13/13, Loss: 2.2571


In [ ]:
import joblib

In [ ]:

joblib.dump(model_classifier, '/content/drive/MyDrive/673Project/ClinicalBert/model_file.pkl')

['/content/drive/MyDrive/673Project/ClinicalBert/model_file.pkl']

In [ ]:
model_classifer = joblib.load('/content/drive/MyDrive/673Project/ClinicalBert/model_file.pkl')

AttributeError: Can't get attribute 'ClinicalBERTClassifier' on <module '__main__'>

In [ ]:
file_path = '/content/drive/MyDrive/673Project/Datasets/IrrelevantSymptoms_100.csv'
df = pd.read_csv(file_path)
print(len(df))
#print(df.head())
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
#print(shuffled_df.head())

100


In [ ]:
original_text = df['Original']
attacks_text = df['Attack']
print(len(original_text))

100


In [ ]:
class AttackDataset(Dataset):
    def __init__(self, original_text, attack_text, tokenizer, max_length=128):
        self.original_texts = original_text
        self.attack_texts = attack_text
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.original_texts)

    def __getitem__(self, idx):
        text = self.original_texts[idx]
        attack = self.attack_texts[idx]
        # Tokenize text
        inputs_original = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        inputs_attack = self.tokenizer(
            attack,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": inputs_original["input_ids"].squeeze(0),
            "attention_mask": inputs_original["attention_mask"].squeeze(0),
            "input_ids_attack": inputs_attack["input_ids"].squeeze(0),
            "attention_mask_attack": inputs_attack["attention_mask"].squeeze(0),
            #"label_index":torch.tensor(label, dtype=torch.long),
        }

#texts = ["The patient shows symptoms of diabetes.", "No signs of hypertension were found."]
#labels = [1, 0]

# Create dataset and DataLoader
#X = list(df['text'])
#Y = df['label']
dataset_attack = AttackDataset(original_text, attacks_text, tokenizer)
print(len(dataset_attack))
dataloader_attack = DataLoader(dataset_attack, batch_size=32, shuffle=True)

100


In [ ]:
#evaluation attacks
def evaluate_model(model_classifier, dataloader_test):
    model_classifier.eval()
    predictions_without_attack, probabilities_without_attack = [], []
    predictions_with_attack, probabilities_with_attack = [], []
    with torch.no_grad():
        for batch in dataloader_attack:
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            input_ids_attack = batch["input_ids_attack"]
            attention_mask_attack = batch["attention_mask_attack"]
            #labels = batch["label"]

            logits_without_attack = model_classifier(input_ids, attention_mask)
            logits_with_attack = model_classifier(input_ids_attack, attention_mask_attack)
            probs_without_attack = F.softmax(logits_without_attack, dim=1)  # Apply softmax to logits
            probs_with_attack = F.softmax(logits_with_attack, dim=1)
            #preds_without = torch.argmax(probs, dim=1)

            probabilities_without_attack.extend(probs_without_attack.numpy())

            probabilities_with_attack.extend(probs_with_attack.numpy())
            #predictions_without_attack.extend(preds.numpy())
            #true_labels.extend(np.argmax(labels.numpy(),axis =1))
            #print(true_labels)
            #sys.exit()
            #true_labels_test = np.argmax(y_test, axis=1)


    #print(len(predictions))
    #print(len(true_labels))
    #accuracy = (predictions == true_labels).sum().item() / true_labels.size(0)
    #accuracy = sum(1 for a, b in zip(predictions, true_labels) if a == b)
    #accuracy = accuracy/len(true_labels)
    #print(accuracy)
    return predictions_without_attack, probabilities_without_attack,predictions_with_attack, probabilities_with_attack

# Evaluate model
predictions_without_attack, probabilities_without_attack,predictions_with_attack, probabilities_with_attack= evaluate_model(model_classifier, dataloader_attack)

In [ ]:
print(len(probabilities_without_attack))

50


In [ ]:
top_k = 15
differences = []
for note_index in range(0,len(probabilities_with_attack)):
   sorted_indices_without_attack = sorted(range(len(probabilities_without_attack[note_index])), key=lambda i: probabilities_without_attack[note_index][i])
   sorted_indices_without_attack.reverse()
   sorted_indices_with_attack = sorted(range(len(probabilities_with_attack[note_index])), key=lambda i: probabilities_with_attack[note_index][i])
   sorted_indices_with_attack.reverse()
   A = set(sorted_indices_without_attack[0:top_k])
   B = set(sorted_indices_with_attack[0:top_k])
   num_matches = len(A.difference(B))
   differences.append(num_matches)

print(np.mean(differences))

1.88


In [ ]:
#evaluation
def evaluate_model(model_classifier, dataloader_test):
    model_classifier.eval()
    predictions, probabilities, true_labels = [], [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            labels = batch["label"]

            logits = model_classifier(input_ids, attention_mask)
            probs = F.softmax(logits, dim=1)  # Apply softmax to logits
            preds = torch.argmax(probs, dim=1)

            probabilities.extend(probs.numpy())
            predictions.extend(preds.numpy())
            true_labels.extend(np.argmax(labels.numpy(),axis =1))
            #print(true_labels)
            #sys.exit()
            #true_labels_test = np.argmax(y_test, axis=1)


    print(len(predictions))
    print(len(true_labels))
    #accuracy = (predictions == true_labels).sum().item() / true_labels.size(0)
    accuracy = sum(1 for a, b in zip(predictions, true_labels) if a == b)
    accuracy = accuracy/len(true_labels)
    print(accuracy)
    return predictions, probabilities, true_labels

# Evaluate model
preds, probs, true_labels = evaluate_model(model_classifier, dataloader_test)

960
960
1.0


In [ ]:
a = [1,0,0,1,1]
b = [0,0,0,1,1]
(a==b)

False

In [ ]:
print(true_labels)

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0.],

In [ ]:
texts = ["The patient shows symptoms of diabetes.", "No signs of hypertension were found."]
labels = [1, 0]  # Example labels

# Tokenize the inputs
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=128)

# Extract inputs for the model
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

In [ ]:
from torch.optim import Adam
import torch

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=2e-5)

# Example training step
model.train()
logits = model(input_ids, attention_mask)
loss = loss_fn(logits, torch.tensor(labels))
loss.backward()
optimizer.step()